In [2]:
import pandas as pd
import numpy as np
import cv2
import os

# 1. Load the CSV file
# Replace 'fer2013.csv' with your actual file name (e.g., 'ckextended.csv')
csv_path = 'ckextended.csv'
df = pd.read_csv(csv_path)

# 2. Define Output Directory
output_dir = "CK_Plus_Images"
os.makedirs(output_dir, exist_ok=True)

# 3. Define Emotion Labels (CK+ standard mapping)
# Check your specific CSV description, but this is the standard CK+ mapping:
emotion_map = {
    0: 'Anger',
    1: 'Disgust',
    2: 'Fear',
    3: 'Happiness',
    4: 'Sadness',
    5: 'Surprise',
    6: 'Neutral',
    7: 'Contempt'
}

print("Starting conversion...")

# 4. Iterate through rows and save images
for index, row in df.iterrows():
    # Get the emotion label
    emotion_label = int(row['emotion'])
    folder_name = emotion_map.get(emotion_label, "Unknown")

    # Create specific emotion folder
    save_path = os.path.join(output_dir, folder_name)
    os.makedirs(save_path, exist_ok=True)

    # Get pixel data (it's usually a space-separated string)
    pixels = np.fromstring(row['pixels'], sep=' ')

    # Reshape to 48x48 image
    image = pixels.reshape(48, 48)

    # Save as image file
    filename = f"image_{index}.png"
    cv2.imwrite(os.path.join(save_path, filename), image)

print(f"Done! Check the '{output_dir}' folder.")

Starting conversion...


[ WARN:0@0.063] global loadsave.cpp:1063 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


Done! Check the 'CK_Plus_Images' folder.


In [3]:
import os
import shutil
import random

# --- CONFIGURATION ---
source_dir = "CK_Plus_Images"   # The folder with your Emotion subfolders
output_dir = "CK_Plus_Ready"    # Where the split data will go
split_ratios = (0.8, 0.1, 0.1)  # Train: 80%, Valid: 10%, Test: 10%
# ---------------------

def split_dataset(source, output, ratios):
    if sum(ratios) != 1.0:
        print("Error: Ratios must sum to 1.0")
        return

    # Create the main output directories
    for split in ['train', 'valid', 'test']:
        os.makedirs(os.path.join(output, split), exist_ok=True)

    # Get list of emotion folders (Anger, Happy, etc.)
    emotions = [d for d in os.listdir(source) if os.path.isdir(os.path.join(source, d))]

    print(f"Found classes: {emotions}")
    print("Splitting files...")

    for emotion in emotions:
        emotion_source_path = os.path.join(source, emotion)

        # Get all images in this emotion folder
        images = [f for f in os.listdir(emotion_source_path)
                  if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        # Shuffle to ensure random selection
        random.shuffle(images)

        # Calculate split counts
        total_images = len(images)
        train_count = int(total_images * ratios[0])
        valid_count = int(total_images * ratios[1])
        # Test gets the rest to ensure no rounding errors miss an image

        # Slicing the list
        train_imgs = images[:train_count]
        valid_imgs = images[train_count : train_count + valid_count]
        test_imgs = images[train_count + valid_count:]

        # Helper function to copy files
        def copy_files(file_list, split_name):
            dest_path = os.path.join(output, split_name, emotion)
            os.makedirs(dest_path, exist_ok=True)
            for img in file_list:
                shutil.copy2(os.path.join(emotion_source_path, img),
                             os.path.join(dest_path, img))

        # Execute copies
        copy_files(train_imgs, 'train')
        copy_files(valid_imgs, 'valid')
        copy_files(test_imgs, 'test')

        print(f"  {emotion}: {len(train_imgs)} train, {len(valid_imgs)} valid, {len(test_imgs)} test")

    print(f"\nSuccess! Data is ready in '{output_dir}'")

# Run the function
split_dataset(source_dir, output_dir, split_ratios)

Found classes: ['Disgust', 'Surprise', 'Anger', 'Sadness', 'Contempt', 'Happiness', 'Fear', 'Neutral']
Splitting files...
  Disgust: 47 train, 5 valid, 7 test
  Surprise: 66 train, 8 valid, 9 test
  Anger: 36 train, 4 valid, 5 test
  Sadness: 22 train, 2 valid, 4 test
  Contempt: 14 train, 1 valid, 3 test
  Happiness: 55 train, 6 valid, 8 test
  Fear: 20 train, 2 valid, 3 test
  Neutral: 474 train, 59 valid, 60 test

Success! Data is ready in 'CK_Plus_Ready'
